In [1]:
%%bash
pip3 install kuzu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 59.6 MB/s eta 0:00:00


In [3]:
import kuzu

db = kuzu.Database("./dblp")
conn = kuzu.Connection(db)

In [8]:
conn.execute("CREATE NODE TABLE Dblp(id STRING, PRIMARY KEY (id))")
conn.execute("CREATE REL TABLE Connected(FROM Dblp TO Dblp)")

In [16]:
conn.execute('COPY Dblp FROM "dblp-nodes.csv"')

In [18]:
conn.execute('COPY Connected from "dblp.csv" (delim="\t")')

In [25]:
%%time

results = conn.execute("MATCH (n1:Dblp)-[:CONNECTED]->(n2:Dblp)-[:CONNECTED]->(n3:Dblp)-[:CONNECTED]->(n4:Dblp)-[:CONNECTED]->(n5:Dblp)-[:CONNECTED]->(n6:Dblp)-[:CONNECTED]->(n7:Dblp) RETURN count(*)")
while results.has_next():
    print(results.get_next())

[179284658061]
CPU times: user 5h 32min 48s, sys: 406 ms, total: 5h 32min 49s
Wall time: 12min 34s


In [15]:
nodeset = set()

with open("snap/noheader/com-dblp.ungraph.txt") as file:
    for line in file:
        splits = line.split("\t")
        nodeset.add(splits[0])
        nodeset.add(splits[1].replace("\n", ""))

str = "\n".join(nodeset)

with open("dblp-nodes.csv", "w") as f:
    f.write(str)